In [ ]:
import pkuseg
import pandas as pd
col=5
seg = pkuseg.pkuseg()           # 以默认配置加载模型
import pickle

In [ ]:
synwords=pickle.load(open('./data_dict/synwords.pickle','rb'))
key_zipped_movie=pickle.load(open('./data_dict/key_zipped_movie.pickle','rb'))
key_zipped_book=pickle.load(open('./data_dict/key_zipped_book.pickle','rb'))
list_zipped_movie=pickle.load(open('./data_dict/list_zipped_movie.pickle','rb'))
list_zipped_book=pickle.load(open('./data_dict/list_zipped_book.pickle','rb'))
movie_raw = pickle.load(open('./data_dict/movie_raw.pickle','rb'))
book_raw = pickle.load(open('./data_dict/book_raw.pickle','rb'))


### 倒排表“运算”函数

In [ ]:
# 合并倒排表
def or_merge(word1_list, word2_list, level=0, jump=0, jump_2=0):
    # level为跳表层数 level=0为不使用跳表 level=1为单层跳表 level=2为双层跳表
    # jump为跳表指针步长
    # jump_2为双层跳表指针步长
    # jump_2应大于jump
    #print(word1, word2)
    #if word1 not in word2doc:
    #    return word2doc[word2]
    #if word2 not in word2doc:
    #    return word2doc[word1]
    if word1_list == []:
        return word2_list
    elif word2_list == []:
        return word1_list
    else:
        # print(word1,word2)
        merge_list = []
        #word1_list = word2doc[word1]
        #word2_list = word2doc[word2]
        print(word1_list)
        print(word2_list)
        i = 0
        j = 0
        # 使用跳表指针
        len1 = len(word1_list)
        len2 = len(word2_list)
        cnt = 0  # 记录比较次数

        # 使用跳表指针，两层跳表
        if level == 2 and jump >= 0 and jump_2 > jump:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    merge_list.append(
                        (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                    i += 1
                    j += 1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    merge_list.append(word1_list[i])
                    i += 1
                    t = i
                    if i+jump_2 < len1 and word1_list[i+jump_2][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump_2
                        while t < i:
                            merge_list.append(word1_list[t])
                            t += 1
                    elif i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump
                        while t < i:
                            merge_list.append(word1_list[t])
                            t += 1
                else:
                    cnt += 1
                    merge_list.append(word2_list[j])
                    j += 1
                    t = j
                    if j+jump_2 < len2 and word2_list[j+jump_2][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump_2
                        while t < j:
                            merge_list.append(word2_list[t])
                            t += 1
                    elif j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump
                        while t < j:
                            merge_list.append(word2_list[t])
                            t += 1
            if (i < len1):
                merge_list.extend(word1_list[i:])
            if (j < len2):
                merge_list.extend(word2_list[j:])

        # 使用跳表指针，单层跳表
        elif level == 1 and jump > 0:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    merge_list.append(
                        (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                    i += 1
                    j += 1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    merge_list.append(word1_list[i])
                    i += 1
                    if i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        t = i
                        i += jump
                        while t < i:
                            merge_list.append(word1_list[t])
                            t += 1
                else:
                    cnt += 1
                    merge_list.append(word2_list[j])
                    j += 1
                    if j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        t = j
                        j += jump
                        while t < j:
                            merge_list.append(word2_list[t])
                            t += 1
            if (i < len1):
                merge_list.extend(word1_list[i:])
            if (j < len2):
                merge_list.extend(word2_list[j:])

        else:
            # 未使用跳表指针
            if level == 0 and jump == 0:
                while i < len1 and j < len2:
                    if word1_list[i][0] == word2_list[j][0]:
                        cnt += 1
                        merge_list.append(
                            (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                        i += 1
                        j += 1
                    elif word1_list[i][0] < word2_list[j][0]:
                        cnt += 1
                        merge_list.append(word1_list[i])
                        i += 1
                    else:
                        cnt += 1
                        merge_list.append(word2_list[j])
                        j += 1
                if (i < len1):
                    merge_list.extend(word1_list[i:])
                if (j < len2):
                    merge_list.extend(word2_list[j:])

        print(cnt)
        return merge_list

In [ ]:
# 合并倒排表
def and_merge(word1_list, word2_list, level=0, jump=0, jump_2=0):
    # level为跳表层数 level=0为不使用跳表 level=1为单层跳表 level=2为双层跳表
    # jump为跳表指针步长
    # jump_2为双层跳表指针步长
    # jump_2应大于jump
    #print(word1, word2)
    #if word1 not in word2doc or word2 not in word2doc:
    #    return []
    if word1_list == [] or word2_list == []:
        return []

    else:
        # print(word1,word2)
        merge_list = []
        #word1_list = word2doc[word1]
        #word2_list = word2doc[word2]
        print(word1_list)
        print(word2_list)
        i = 0
        j = 0
        # 使用跳表指针
        len1 = len(word1_list)
        len2 = len(word2_list)
        cnt = 0  # 记录比较次数

        # 使用跳表指针，两层跳表
        if level == 2 and jump >= 0 and jump_2 > jump:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    merge_list.append(
                        (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                    i += 1
                    j += 1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    i += 1
                    if i+jump_2 < len1 and word1_list[i+jump_2][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump_2
                    elif i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump
                else:
                    cnt += 1
                    j += 1
                    if j+jump_2 < len2 and word2_list[j+jump_2][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump_2
                    elif j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump

        # 使用跳表指针，单层跳表
        elif level == 1 and jump > 0:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    merge_list.append(
                        (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                    i += 1
                    j += 1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    i += 1
                    if i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump
                else:
                    cnt += 1
                    j += 1
                    if j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump

        else:
            # 未使用跳表指针
            if level == 0 and jump == 0:
                while i < len1 and j < len2:
                    if word1_list[i][0] == word2_list[j][0]:
                        cnt += 1
                        merge_list.append(
                            (word1_list[i][0], word1_list[i][1]+word2_list[j][1]))
                        i += 1
                        j += 1
                    elif word1_list[i][0] < word2_list[j][0]:
                        cnt += 1
                        i += 1
                    else:
                        cnt += 1
                        j += 1

        print(cnt)
        return merge_list

In [ ]:
def not_merge( word2_list, word1_list, level=0, jump=0, jump_2=0):
    # 形式为 word1 not word2
    # word1默认为空
    # level为跳表层数 level=0为不使用跳表 level=1为单层跳表 level=2为双层跳表
    # jump为跳表指针步长
    # jump_2为双层跳表指针步长
    # jump_2应大于jump

    #if word1 != '' and word1 not in word2doc:
    #    return []
    #if word1 != '' and word2 not in word2doc:
    #    return word2doc[word1]
    #else:
    #    if word1 == '':
    #        word1_list=[(i,0) for i in range(0,len(word2doc))]
    #        if word2 not in word2doc:
    #            return word1_list
    #    else:
    #        word1_list = word2doc[word1]
    if word1_list == []:
        return []
    elif word2_list == []:
        return word1_list
    else:
        # 只剩下word2在倒排表中的情况
        #word2_list = word2doc[word2]
        print(word1_list)
        print(word2_list)
        i = 0
        j = 0
        # 使用跳表指针
        len1 = len(word1_list)
        print(len1)
        len2 = len(word2_list)
        cnt = 0  # 记录比较次数
        # 使用跳表指针，两层跳表
        if level == 2 and jump >= 0 and jump_2 > jump:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    word1_list.pop(i)
                    len1-=1
                    j+=1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    i += 1
                    if i+jump_2 < len1 and word1_list[i+jump_2][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump_2
                    elif i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump
                else:
                    cnt += 1
                    j += 1
                    if j+jump_2 < len2 and word2_list[j+jump_2][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump_2
                    elif j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump

        # 使用跳表指针，单层跳表
        elif level == 1 and jump > 0:
            while i < len1 and j < len2:
                if word1_list[i][0] == word2_list[j][0]:
                    cnt += 1
                    word1_list.pop(i)
                    len1-=1
                    j+=1
                elif word1_list[i][0] < word2_list[j][0]:
                    cnt += 1
                    i += 1
                    if i+jump < len1 and word1_list[i+jump][0] <= word2_list[j][0]:
                        cnt+=1
                        i += jump
                else:
                    cnt += 1
                    j += 1
                    if j+jump < len2 and word2_list[j+jump][0] <= word1_list[i][0]:
                        cnt+=1
                        j += jump
        else:
            # 未使用跳表指针
            if level == 0 and jump == 0:
                while i < len1 and j < len2:
                    if word1_list[i][0] == word2_list[j][0]:
                        cnt+=1
                        word1_list.pop(i)
                        len1-=1
                        j+=1
                    elif word1_list[i][0] < word2_list[j][0]:
                        cnt += 1
                        i += 1
                    else:
                        cnt += 1
                        j += 1

        print(cnt)
        return word1_list

### 处理query

In [ ]:
def stopwords_query(filepath,query):
    lenth=len(query)
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = f.read()
        # 删去query中的停用词
    i=0
    while(i<lenth):
        # 若字符在停用表中
        if query[i] in stopwords:
            query.remove(query[i])
            lenth-=1
        else:
            i+=1
    return query

In [ ]:
def findsyn(word, synwords):
    if len(word) == 1:
        return -1
    for line in synwords:
        if word in line:
            return line
    return -1

In [ ]:
def word_syn(word,synwords):
    #将近义词替换为同义词列表第一个词
    #遍历分词、去停用词后的结果
    if findsyn(word, synwords) != -1:
        word = findsyn(word, synwords)[0]
    return word

In [ ]:
# 处理查询语句
# 对query分词
def query_seg(query):
    query = seg.cut(query)
    # 去停用词
    query = stopwords_query('./stopwords-master/hit_stopwords.txt', query)
    query = stopwords_query('./stopwords-master/baidu_stopwords.txt', query)
    bool_query=""
    for word in query:
        bool_query+=word_syn(word,synwords)
        bool_query+=" AND "
    bool_query=bool_query[:-5]
    print(query)
    print(bool_query)
    return bool_query

In [ ]:
import re

def process_query(query):
    stack = []  # 用于存储括号的栈
    output = []  # 用于存储处理后的查询表达式
    query=query.upper()
    query=query.replace('AND',' AND ')
    query=query.replace('OR',' OR ')
    query=query.replace('NOT',' NOT ')
    query=query.replace('（',' ( ')
    query=query.replace('）',' ) ')
    query=re.sub(r"\s+", " ", query)

    query=query.split(' ')
    print(query)
    query_s = ""
    for phrase in query:
        if phrase in ["AND", "OR", "NOT", "(", ")"]:
            query_s+=phrase
            query_s+=" "
        else:
            #同义词替换
            query_s+=query_seg(phrase)
            query_s+=" "
    print(query_s)
    query_s=re.sub(r"\s+", " ", query_s)

    # 使用正则表达式拆分查询语句
    tokens = re.split(r"(\(|\)|\bAND\b|\bOR\b|\bNOT\b)", query_s)

    for token in tokens:
        token = token.strip()
        
        if token == "":
            continue

        if token == "(":
            stack.append(token)
        elif token == ")":
            while stack and stack[-1] != "(":
                output.append(stack.pop())
            
            if stack and stack[-1] == "(":
                stack.pop()  # 弹出左括号
        else:
            if token in ["AND", "OR", "NOT"]:
                while stack and stack[-1] != "(" and precedence(stack[-1]) >= precedence(token):
                    output.append(stack.pop())
                
                stack.append(token)
            else:
                output.append(token)

    # 将栈中剩余的操作符弹出并添加到输出中
    while stack:
        output.append(stack.pop())

    return output

# 定义运算符的优先级
def precedence(operator):
    if operator == "NOT":
        return 3
    elif operator == "AND":
        return 2
    elif operator == "OR":
        return 1
    else:
        return 0

In [ ]:
#在压缩字典中查找
def find_zipped(key_zipped,list_zipped,word):
    index=0
    if word in key_zipped:
        while(index<len(key_zipped)):
            index=key_zipped[index+1:].find(word)+index+1#第一次出现的位置
            print(word,index)
            if index in list_zipped:
                #print('--------------------')
                return list_zipped[index]
    else:
        return []
    

In [ ]:
def perform_query(operand1, operand2, operator, level=2, jump=5, jump_2=10):
    # 在这里执行实际的查询操作
    if operator == "AND":
        return and_merge(operand1, operand2,level,jump,jump_2)
    elif operator == "OR":
        return or_merge(operand1, operand2,level,jump,jump_2)
    elif operator == "NOT":
        return not_merge(operand2, operand1,level,jump,jump_2)

def evaluate_query(key_zipped,list_zipped,postfix_expression):
    stack = []  # 用于计算后缀表达式的栈

    for i in range (0,len(postfix_expression)):
        token = postfix_expression[i]
        if token in ["AND", "OR"]:
            operand2 = stack.pop()
            print(operand2)
            operand1 = stack.pop()
            print(operand1)
            if(type(operand1) == str):
                try:
                    operand1 = find_zipped(key_zipped,list_zipped,operand1)
                except:
                    operand1=[]
            print(operand1)
            if(type(operand2)==str):
                #try:
                operand2 = find_zipped(key_zipped,list_zipped,operand2)
                #except:
                   # operand2=[]
            print(operand2)
            result = perform_query(operand1, operand2, token)
            stack.append(result)
        elif token == "NOT":
            operand2 = stack.pop()
            print(operand2)
            if(type(operand2)==str):
                try:
                    operand2 = find_zipped(key_zipped,list_zipped,operand2)
                except:
                    operand2=[]
            #看下一个token是不是AND或者OR
            if i+1<len(postfix_expression):
                if postfix_expression[i+1] == "AND":
                    i+=1
                    operand1 = stack.pop()
                    if(type(operand1) == str):
                        try:
                            operand1 = find_zipped(key_zipped,list_zipped,operand1)
                        except:
                            operand1=[]
                    print(operand1)
                elif postfix_expression[i+1] == "OR":
                    operand1 = [(j,0) for j in range(0,len(list_zipped))]
            elif len(stack)==0:#栈中只有一个元素，已经作为operand2弹出
                operand1 = [(j,0) for j in range(0,len(list_zipped))]
            else:
                #print(len(stack))
                operand1 = stack.pop()
                if(type(operand1) == str):
                    try:
                        operand1 = find_zipped(key_zipped,list_zipped,operand1)
                    except:
                        operand1=[]
                print(operand1)
            
            result = perform_query(operand1, operand2, token)
            stack.append(result)    
        else:
            stack.append(token)

    #return stack[0] if stack else None
    if type(stack[0])==str:
#        try:
        stack[0]=find_zipped(key_zipped,list_zipped,stack[0])
#            print(111)
#        except:
#            stack[0]=[]
#            print(222)
    
    return stack[0] if stack else None

evaluate_query(key_zipped_movie,list_zipped_movie,"活着")

In [ ]:
print("搜索电影请输入'1',搜索书籍请输入'2'：")
choice = input()
if choice == '1':
    key_zipped=key_zipped_movie
    list_zipped=list_zipped_movie
    data = movie_raw
else:
    key_zipped=key_zipped_book
    list_zipped=list_zipped_book
    data = book_raw

print("请输入query：")
query = input()
processed_query = process_query(query)
print(processed_query)
result = evaluate_query(key_zipped,list_zipped, processed_query)
result.sort(key=lambda x:x[1],reverse=True)
print("result=", result)
for i in range(0, len(result)):
    print(data[result[i][0]])